## Lab 4
### UGBA 88: Data and Decisions, Fall 2019


---


<img src='michigan_map.png'>

## The Effectiveness of a 'Get-Out-The-Vote' Campaign 

In this lab we will go over the matching method for causal infererence.

This lab is designed to be completed in class. However, in case you need additional time, this assignment is due **Tuesday, November 5th at 11:59pm**.

The lab will be graded for **completion**. Lab office hours are held by Connector Assistants on Tuesdays after labs from 2-4pm in the DS Nexus in Moffitt.

### Table of Contents

[Background](#bground) <br>
1. [The Dataset](#data) <br>
2. [Matching on a Single Covariate](#single) <br>
3. [Matching on a Set of Covariates](#set) <br>


In [ ]:
from datascience import *
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings('ignore')

from client.api.notebook import Notebook
ok = Notebook('lab4.ok')
_ = ok.auth(inline=True)

**Warm-up:** In your own words, what is the **fundamental problem of causal inference**?

*#write answer here*

----

## Background <a id='bground'></a>

Compared to other developed countries, the United States has low voter turnout rates. About 55.5% of the U.S. voting-age population casted ballots in the 2016 presidential election. In the 2000 presidential election, the turnout rate was 50.3%.

Fearing another low turnout in the 2002 midterm elections, civic groups in Iowa and Michigan launched the Vote 2002 Campaign to boost voter turnout. In the week prior to the election, Vote 2002 volunteers called 60,000 voters on the phone and gave them the following message:

*"Hello, may I speak with [name of person] please? Hi. This is [caller's name] calling from Vote 2002, a nonpartisan effort working to encourage citizens to vote. We just wanted to remind you that elections are being held this Tuesday. The success of our democracy depends on whether we exercise our right to vote or not, so we hope you'll come out and vote this Tuesday. Can I count on you to vote next Tuesday?”*

Our causal question for the day is: **Did the Vote 2002 campaign work? Did it increase voter turnout in the 2002 Congressional elections?** 

To estimate the causal effect of receving a Vote 2002 phone call, we'll need to compare the outcomes of voters that received a call--we'll call them **contacted** voters--to the outcomes for some comparison group. For our causal effect estimate to be accurate, the comparison group we use will need to reflect the *counterfactual* outcomes for contacted voters: what those voters *would have* done if they had not received the Vote 2002 call. If the comparison group poorly represents the counterfactual, our estimate will be biased.

For the comparison group to represent the counterfactual, we need the only relevant difference between contacted voters and comparison voters to be that comparison voters did not receive a phone call while contacted voters did. The idea of **matching** is to find a comparable uncontacted voter for each contacted voter, and then use that set of matches as a comparison group.

----

## Section 1: The Dataset <a id='data'></a>

The dataset we'll use was compiled by the Vote 2002 campaign staff. The staff had demographic data for each voter they attempted to contact, and marked whether each call was completed successfully or not. Later, in order to measure the impact of their campaign, they merged these data with official voting records to see if voters did go out and vote.

Here is a description of each column in the dataset:

* **contact** - indicator for whether voter was successfully contacted by volunteer
* **vote02** - indicator for whether the voter votes in the 2002 election (*this is the outcome of interest*)
* **vote98** - indicator for whether the voter voted in the 1998 election
* **newreg** - indicator for wheter voter is newly registered voter
* **age** -  age of voter
* **female** - indicator for female

The original data include over 2 million potential voters in Iowa and Michigan. In order for us to not spend the whole lab period waiting for a cell to finish running, we'll focus on a subset of the data: voters from Oakland county in Michigan (the county that contains Detroit).

In [ ]:
#run this cell to load the data

#read in data
data = Table.read_table("mi_voters.csv")

data.show(5)

**Q1.1:** Now for a coding warm-up. What percentage of voters in our data were *contacted*?

In [ ]:
#assign the percentage to the variable percent_voters_contacted

percent_voters_contacted = ...
percent_voters_contacted

In [ ]:
_ = ok.grade("q1_1")

Let's first check how voter turnout compares between contacted and non-contacted voters. For exercises below we'll find it useful to separate the data into two tables, one for contacted voters and the other for non-contacted voters.

In [ ]:
#defines separate tables for contacted and non-contacted
contacted = data.where('contact', 1)
noncontacted = data.where('contact', 0)

Next we'll calculate the turnout for each group of voters.

In [ ]:
vote02_contacted = 100*np.mean(contacted.column('vote02'))
vote02_noncontacted = 100*np.mean(noncontacted.column('vote02'))

print('2002 Election Turnout Rate, Contacted Voters:', vote02_contacted)
print('2002 Election Turnout Rate, Non-Contacted Voters:', vote02_noncontacted)

Finally let's plot the results in a bar plot.

In [ ]:
turnout_rates = Table().with_columns('group', ['contacted', 'non-contacted'],
                                       '2002 Turnout Rate (%)', [vote02_contacted, vote02_noncontacted])
turnout_rates.barh('group')

Indeed, contacted voters have a 13 percentage point higher turnout in the 2002 election.

Does this difference reflect the causal effect of receiving a call? It might. But it might not if contacted and not contacted voters are different in ways other than whether they received a call.

As a starting point, let's compare the age distributions for the two groups of voters.

In [ ]:
data.hist('age', group = 'contact', bins = 20)

**Q1.2:** Why might the chart above suggest that simply comparing contacted and non-contacted voters will lead to a *biased* estimate for the average causal effect of receiving a call?

*#Write your answer here*

----

## Section 2: Matching on a Single Covariate <a id='single'></a>

To account for these differences in age across the two voter populations, let's try to *'control for'* age via matching. That is, for each contacted voter, we'll find a non-contacted voter with the same age, and use the set of matched non-contacted voters as our comparison group.

First let's assign ID's to people in the control group. It will soon become clear why we're doing this.

In [ ]:
#add ID column
noncontacted = noncontacted.with_column('id', np.arange(noncontacted.num_rows))
noncontacted.show(5)

**Q2.1:** Write a function that takes in an `age` value and returns the ID of a **randomly chosen** non-contacted adult with the given age.

In [ ]:
#write your code here
def match_age_id(age_value):
    ...

**Q2.2:** Let's test the function you've written. Find an age match for the first row in the `contacted` table.

In [ ]:
#run this cell to observe first row from `contacted` table
contacted.take(0)

In [ ]:
#run your function
match_age_id(...)

In [ ]:
#confirm your match has the same age

**Q2.3:** How many potential age matches could you have chosen?

In [ ]:
#write code here

**Q2.4** Use your function `match_age_id` to create a table of non-contacted voters (called `noncontacted_age_matches`) that corresponds to the matches for each row in the `contacted` table. (Hint: remember `.take`). **The code may take a couple minutes to run**, so feel free to answer the next question (**Q2.5**) while the notebook executes the code.

If you're rusty on how to use the `.apply` and `.take`, you may find the [Data 8 Python reference page](http://data8.org/fa18/python-reference.html) helpful.

In [ ]:
#use .apply to get array of matched noncontacted voters
match_indices = ...

#use .take and `match_indices` to get a table of matched noncontacted voters
noncontacted_age_matches = ...
noncontacted_age_matches


In [ ]:
_ = ok.grade("q2_4")

**Q2.5:** For your matching function, what is the downside of selecting *one random row* for matching? How could we utilize data from all matched rows?

*#write answer here*

**Q2.6:** Compare the age distributions of `contacted` voters to *matched* `noncontacted` voters. What do you notice?

In [ ]:
#plot histogram for `contacted` voters

In [ ]:
#plot histogram for `noncontacted_age_matches` voters

*#What do you notice?*

We have controlled for age via matching. Let's see how this changes our estimate for the effect of receiving a Vote 2002 call on turnout. We'll compare 2002 turnout rates for our three groups of voters. [*Note that because your matching procedure has a random component, your results will not match your neighbor's results exactly.*]

In [ ]:
#same as above, just adding turnout rate for noncontacted matches (matched on age)
vote02_contacted = 100*np.mean(contacted.column('vote02'))
vote02_noncontacted = 100*np.mean(noncontacted.column('vote02'))
vote02_noncontacted_age_matches = 100*np.mean(noncontacted_age_matches.column('vote02'))

print('2002 Election Turnout Rate, Contacted Voters:', vote02_contacted)
print('2002 Election Turnout Rate, Non-Contacted Voters:', vote02_noncontacted)
print('2002 Election Turnout Rate, Age Matched Non-Contacted Voters:', vote02_noncontacted_age_matches)

In [ ]:
turnout_rates = Table().with_columns('group', ['contacted', 'non-contacted', 'non-contacted matches'],
                                       '2002 Turnout Rate (%)', [vote02_contacted, vote02_noncontacted, vote02_noncontacted_age_matches])
turnout_rates.barh('group')

**Q2.7:** How do the matched (on age) non-contacted voters compare?

*#write your answer here*

We've controlled for age, but what about the other covariates available to us? How do they compare across our three groups? Let's examine `vote98`, and indicator for whether the voter voted in the 1998 congressional election.

**Q2.8:** Adopting the code from the two code cells above, compare the *1998* turnout rates for our three groups. 

In [ ]:
#print 1998 turnout rates for three groups

In [ ]:
#make bar plot here

**Q2.9:** How do these turnout rates compare? Are our non-contacted matches a good comparison group? (Remember that 1998 turnout is a *covariate* in this context, not our outcome of interest.)

*#write answer here*

## Section 3: Matching on a Set of Covariates <a id='set'></a>

Matching on `age` alone produces a comparison group that is still unbalanced on some dimensions. Let's try matching on all the covariates available to us.

**Q3.1:** Extend the function you wrote in **Q2.1** to match on four covariates: `age`, `newreg`, `female`, and `vote98`.

The function should take as input a value for each covariate and returns the ID of a randomly chosen non-contacted adult with the same values for those covariates.

In [ ]:
#write your code here
def match_full_id(age_value, newreg_value, female_value, vote98_value):
    ...

**Q3.2:** As we did before, let's test the new function you've written. Find a match for the first row in the `contacted` table.

In [ ]:
contacted.take(0)

In [ ]:
#run your function

In [ ]:
#confirm your match has the same values for: `age`, `newreg`, `female`, and `vote98`

**Q3.3:** As you did in **Q2.4**, Use `match_full_id` to create a table of non-contacted voters (called `noncontacted_full_matches`) that corresponds to the matches for reach row in the `contacted` table. **The code may take a couple minutes to run.**

In [ ]:
full_match_indices = ...
noncontacted_full_matches = ...
noncontacted_full_matches

In [ ]:
_ = ok.grade("q3_3")

**Q3.4:** Check for balance between `contacted` and matched `noncontacted` for *one* of these covariates: `newreg`, `female`, or `vote98`.

In [ ]:
#print the mean of covariate for both groups

Finally, let's look at how 2002 turnout rates look like in our new and improved matched comparison group.

In [ ]:
#calculate turnout rates
vote02_contacted = 100*np.mean(contacted.column('vote02'))
vote02_noncontacted = 100*np.mean(noncontacted.column('vote02'))
vote02_noncontacted_age_matches = 100*np.mean(noncontacted_age_matches.column('vote02'))
vote02_noncontacted_full_matches = 100*np.mean(noncontacted_full_matches.column('vote02'))

#print turnout rates
print('2002 Election Turnout Rate, Contacted Voters:', vote02_contacted)
print('2002 Election Turnout Rate, Non-Contacted Voters:', vote02_noncontacted)
print('2002 Election Turnout Rate, Age Matched Non-Contacted Adults:', vote02_noncontacted_age_matches)
print('2002 Election Turnout Rate, Fully Matched Non-Contacted Adults:', vote02_noncontacted_full_matches)

In [ ]:
turnout_rates = Table().with_columns('group', ['contacted', 'non-contacted', 'non-contacted age matches', 'non-contacted full matches'],
                                       '2002 Turnout Rate (%)', [vote02_contacted, vote02_noncontacted, vote02_noncontacted_age_matches, vote02_noncontacted_full_matches])
turnout_rates.barh('group')

**Q3.5:** Using our matched comparison group, what would you estimate for the average causal effect of receiving a Vote 2002 call on turnout?

*#write your answer here*

Finally, we will test whether the difference in turnout rates between contacted voters and matched noncontacted voters that we measured above is *statistically significant*. As in the previous lab, we will use a *permutation test* to answer this question. We'll use the function `permuted_sample_average_difference`.

In [ ]:
#Run this cell to define the function permuted_sample_average_difference 
def permuted_sample_average_difference(table, label, group_label, repetitions):

    tbl = table.select(group_label, label)

    differences = make_array()
    for i in np.arange(repetitions):
        shuffled = tbl.sample(with_replacement = False).column(1)
        original_and_shuffled = tbl.with_column('Shuffled Data', shuffled)

        shuffled_means = original_and_shuffled.group(group_label, np.average).column(2)
        simulated_difference = shuffled_means.item(1) - shuffled_means.item(0)

        differences = np.append(differences, simulated_difference)

    return differences

Run the cell below to conduct a permutation test. **This may take a couple minutes to run.**

In [ ]:
##run this cell to conduct permutation test

#create table that combines `contacted` and `noncontated_full_matches`
contact = np.concatenate((contacted.column('contact'), noncontacted_full_matches.column('contact')))
vote02 = np.concatenate((contacted.column('vote02'), noncontacted_full_matches.column('vote02')))

combined = Table().with_columns('contact', contact, 'vote02', vote02)

#calculate different in 2002 voter turnout
vote02_diff = combined.where('contact', 1).column('vote02').mean() - combined.where('contact', 0).column('vote02').mean()

#assign array of simulated test statistics under null
vote02_sim_diff = permuted_sample_average_difference(combined, 'vote02', 'contact', 10000)

#assign p-value
vote02_pvalue = np.count_nonzero(abs(vote02_sim_diff) >= abs(vote02_diff))/10000

#print results
print('estimated treatment effect =', 100*vote02_diff)
print('p_value =', vote02_pvalue)


**Q3.6:** Are you convinced that the Vote 2002 mobilization effor increased voter turnout? Why or why not?

*#write your answer here*

----
Congratulations, you've finished Lab 4! To submit the lab, run the two cells below:

In [ ]:
# For your convenience, you can run this cell to run all the tests at once
import os
_ = [ok.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('q')]

In [ ]:
_ = ok.submit()